# Part 1 - Exploring Sub Wallet Management

This agent has been initialised using the multitenant flag. This means a single ACA-Py instance can be used to manage multiple subwallets. Each tenant gets their own encrypted data storage for managing their own connections, credentials and interactions etc. 

A mutli-tenant ACA-Py instance contains a base wallet that is only capable of managing the creation and deletion of subwallets. Subwallets then must authenticate to the ACA-Py agent using a JWT Token generated when the subwallet is created. A tenant agent can do all funcationality of a standard ACA-Py instance.

### Useful links

* [What is mult-tenancy](https://whatis.techtarget.com/definition/multi-tenancy)
* [ACA-Py mult-tenant documentation](https://github.com/hyperledger/aries-cloudagent-python/blob/main/Multitenancy.md)



### Tutorial Structure

1. Create a subwallet for Alice (this notebook)
2. Authenticate as Alice using the tenant_jwt and configure a mediator
3. Issue Alice a Credential from an External Agent
4. Alice Issues a Credential to the External Agent

### Initialise the multitenant agent controller

In [ ]:
%autoawait
import time
import asyncio
import pprint

from aries_basic_controller.aries_controller import AriesAgentController
from aries_basic_controller.aries_tenant_controller import AriesTenantController

# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://multitenant-agent:8021"

In [ ]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key="password", is_multitenant=True)

In [ ]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT, webhook_base=WEBHOOK_BASE)

In [ ]:
await agent_controller.listen_webhooks()

### Check for subwallets on the agent

This should yield an empty result, but not error. That means we successfully asked the base wallet of the multitenant agent about subwallets it has stored. ACA-Py instances not in the multitenant configuration will not have access to this API.

In [ ]:
subwallets = await agent_controller.multitenant.query_subwallets()
pp.pprint(subwallets)


In [ ]:
# Uncomment and run the following to remove ALL existing subwallets

# for i in range(len(subwallets['results'])):
#     await agent_controller.multitenant.remove_subwallet_by_id(subwallets['results'][i]['settings']['wallet.id'])

### Let's create a subwallet for Alice

Below is an example payload to achieve this. These properties should be fairly familiar to you if you have been through the other tutorials and looked in the manage file before.

A key different is the `key_management_mode` which is this case is set to `managed`. The base wallet, is managing the keys of Alice's agent. Alice is dependent on them to do this trustworthily

In [ ]:
## First let's create the payload

payload = {
  "image_url": "https://aries.ca/images/sample.png",
  "key_management_mode": "managed",
  "label": "Alice",
  "wallet_dispatch_type": "default",
  "wallet_key": "MySecretKey1234",
  "wallet_name": "AlicesWallet",
  "wallet_type": "indy",
}

In [ ]:
## Now, we create the wallet on the agent 
response_alice = await agent_controller.multitenant.create_subwallet(payload)
pp.pprint(response_alice)

### Let's create another wallet for Joe

Note, that here we have changed the `label` and the `wallet_name` values. The `wallet_name` has to be unique. If you were to try and create another subwallet with the same wallet name, you would receive an error, because wallet names are unique identifiers.

In [ ]:
## First let's create the payload

payload = {
  "image_url": "https://aries.ca/images/sample.png",
  "key_management_mode": "managed",
  "label": "Joe",
  "wallet_dispatch_type": "default",
  "wallet_key": "MySecretKey123",
  "wallet_name": "JoesWallet2",
  "wallet_type": "indy",
}

In [ ]:
## Now, we create the wallet on the agent 

response_joe = await agent_controller.multitenant.create_subwallet(payload)
pp.pprint(response_joe)

### Extract the wallet ID

The wallet id is a unique identifier created by the ACA-Py instance to identify a particular tenant wallet instance. The base wallet controller can use this when interacting with subwallets through the multitenant API.

In [ ]:
wallet_id_alice = response_alice['wallet_id']
wallet_id_joe = response_joe['wallet_id']

print("Alice's ID: " + wallet_id_alice)
print("Joe's ID: "  + wallet_id_joe)


### Update a single subwallet

Let's update Joe's wallet label to Joeseph via the controller

In [ ]:
request_body = {
  "wallet_webhook_urls": [
    f"http://multitenant-notebook:8022/{wallet_id_alice}"
  ]
}

response = await agent_controller.multitenant.update_subwallet_by_id(request_body, wallet_id_alice)
pp.pprint(response)

In [ ]:
request_body = {
  "wallet_webhook_urls": [
    f"http://multitenant-notebook:8022/{wallet_id_joe}"
  ]
}

response = await agent_controller.multitenant.update_subwallet_by_id(request_body, wallet_id_joe)
pp.pprint(response)

In [ ]:
response_all_wallets = await agent_controller.multitenant.query_subwallets()
# response_single_wallet = await agent_controller.multitenant.get_single_subwallet_by_id(wallet_id)

# print(response_single_wallet)
pp.pprint(response_all_wallets)

### Get the auth token for a  subwallet

Subwallets have unique authentication tokens that can be obtained via the controller

In [ ]:
response_alice = await agent_controller.multitenant.get_subwallet_authtoken_by_id(wallet_id_alice)
response_joe = await agent_controller.multitenant.get_subwallet_authtoken_by_id(wallet_id_joe)

## Store Alice's JWT for use in later tutorials

The % is a magic method to store variables from a notebook to the jupyter runtime

In [ ]:
alice_jwt = response_alice["token"]
joe_jwt = response_joe["token"]

print("Alice JWT", alice_jwt)
print("Joe JWT", joe_jwt)

In [ ]:
%store alice_jwt
%store wallet_id_alice

In [ ]:
# Based on the aca-py agent you wish to control
alice_agent_controller = AriesTenantController(admin_url=ADMIN_URL, wallet_id=wallet_id_alice, tenant_jwt=alice_jwt)

In [ ]:
joe_agent_controller = AriesTenantController(admin_url=ADMIN_URL, wallet_id=wallet_id_joe, tenant_jwt=joe_jwt)

In [ ]:
def connection_handler(payload):
    print("Alices Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

def messages_handler(payload):
    print("Alices Recieved a Message")
    connection_id = payload["connection_id"]

    print("Handle message", payload, connection_id)
    
message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

alice_agent_controller.register_listeners([message_listener,connection_listener], defaults=True)

In [ ]:
def joe_connection_handler(payload):
    print("Joes Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
joe_connection_listener = {
    "handler": joe_connection_handler,
    "topic": "connections"
}

def joe_messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(joe_agent_controller.messaging.send_message(connection_id, "This is a response from Joe"))
    print("Handling Joes message", payload, connection_id)


joe_message_listener = {
    "handler": joe_messages_handler,
    "topic": "basicmessages"
}


joe_agent_controller.register_listeners([joe_message_listener,joe_connection_listener], defaults=True)

In [ ]:
response = await alice_agent_controller.connections.create_invitation()
print(response["invitation"])
joe_connection_id = response["connection_id"]

joes_invitation = response["invitation"]

In [ ]:
response = await joe_agent_controller.connections.accept_connection(joes_invitation)

In [ ]:
await alice_agent_controller.messaging.send_message(joe_connection_id, "Hey Joe, it's Alice here")

### Remove the subwallet from the agent 

We can easily use the controller to remove the subwallets. Note that Joe has no immediate control over the decision of the base wallet controller to remove their wallet. Again Joe trusts the Base controller of the multitenant agent to manage his wallet in good faith, or in line with some contractual agreement etc.

Let's go ahead and remove Joe's wallets from the base wallet.

In [ ]:
response_joe = await agent_controller.multitenant.remove_subwallet_by_id(wallet_id_joe)

pp.pprint(response_joe)

### Check Joe's wallet has been removed

This should now give a result only containing Alice's wallet

In [ ]:
response_all_wallets = await agent_controller.multitenant.query_subwallets()
pp.pprint(response_all_wallets)

### Terminate the controllers

Let's alos terminate the controller.

In [ ]:
response = await agent_controller.terminate()
print(response)

In [ ]:
await alice_agent_controller.terminate()
await joe_agent_controller.terminate()

### Continue to [Part 2](http://localhost:8888/lab/tree/Alice/Part%202%20-%20Mediation%20of%20communication%20-%20Alice.ipynb) where you will learn how to interact with the ACA-Py instance as Alice